**Imports & Setup**

In [1]:
import os
import litellm
from dotenv import load_dotenv

# 1. Disable CrewAI Telemetry (Prevents hanging)
os.environ["CREWAI_TELEMETRY_OPT_OUT"] = "true"

# 2. Disable LiteLLM background threads (Prevents hanging)
litellm.telemetry = False
litellm.success_callback = []
litellm.failure_callback = []

# 3. Import CrewAI components
from crewai import Agent, Task, Crew, LLM
from crewai.tools import BaseTool

# 4. Load environment variables
load_dotenv()

True

**Initialize Groq LLM**

In [2]:
# Initialize Groq (Llama 3.3)
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.1
)

# Quick connection test
print("Testing connection...")
print(f"Response: {llm.call('Hello')}")

Testing connection...
Response: Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?


**Define Custom Tool**

In [3]:
class ReplaceJargonsTool(BaseTool):
    name: str = "Jargon replacement tool"
    description: str = "Replaces jargon with more specific terms."

    def _run(self, email: str) -> str:
        replacements = {
            "PRX": "Project Phoenix (internal AI revamp project)",
            "TAS": "technical architecture stack",
            "DBX": "client database cluster",
            "SDS": "Smart Data Syncer",
            "SYNCBOT": "internal standup assistant bot",
            "WIP": "in progress",
            "POC": "proof of concept",
            "ping": "reach out"
        }
        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing '{jargon}' with '{replacement}'")

        return "\n".join(suggestions) if suggestions else "No jargon or internal abbreviations detected."

# Instantiate the tool
jt = ReplaceJargonsTool()

# Test the tool in isolation
original_email = """looping in Sharon. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.
Let's sync up tomorrow if SYNCBOT allows 😄. ping me if any blockers."""

print("--- Tool Test Output ---")
print(jt.run(original_email))

--- Tool Test Output ---
Using Tool: Jargon replacement tool
Consider replacing 'PRX' with 'Project Phoenix (internal AI revamp project)'
Consider replacing 'TAS' with 'technical architecture stack'
Consider replacing 'SDS' with 'Smart Data Syncer'
Consider replacing 'SYNCBOT' with 'internal standup assistant bot'
Consider replacing 'ping' with 'reach out'


**Define Agent & Task**

In [4]:
# Create the Agent
email_assistant = Agent(
    role="Email Assistant Agent",
    goal="Improve emails and make them sound professional and clear",
    backstory="A highly experienced communication expert skilled in professional email writing",
    verbose=True,
    tools=[jt],     # <--- Giving the agent the tool here
    llm=llm,
    max_rpm=10      # <--- Rate limit for Groq Free Tier
)

# Create the Task
email_task = Task(
    description=f"""Take the following rough email and rewrite it into a professional and polished version.
    Expand abbreviations:
    '''{original_email}'''""",
    agent=email_assistant,
    expected_output="A professional written email with proper formatting and content."
)

# Assemble the Crew
crew = Crew(
    agents=[email_assistant],
    tasks=[email_task],
    verbose=True
)

**Execute Workflow**

In [ ]:
result = crew.kickoff()

print("\n\n########################")
print("## Final Email Draft ##")
print("########################\n")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1735666c-fffb-43a2-8843-53a278b2790c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Task: Take the following rough email and rewrite it into a professional and polished version.                  │
│      Expand abbreviations:                                                                                      │
│      '''looping in Sharon. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.              │
│  Let's sync up tomorrow if SYNCBOT allows 😄. ping me if any blockers.'''                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

Output()

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8ebaf8bf-1388-4919-b9b5-80cb7d057965                                                                     │
│  Agent: Email Assistant Agent                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯